In [131]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import FeatureHasher
from funciones_varias import cargaCsvToDataFrame

dfMoviesFinal = cargaCsvToDataFrame("dfMoviesFinal", "datasets_eda")

In [132]:
def hashEncoding(mainDF, column, numBins):
    lista= list(mainDF[column].unique())
    data={column:[[i]for i in lista]}

    df = pd.DataFrame(data)

    hasher = FeatureHasher(n_features=numBins, input_type='string')

    hashed_features = hasher.transform(df[column])

    hashed_df = pd.DataFrame(hashed_features.toarray(), columns=[f"hashed_{i}" for i in range(numBins)])
    
    return hashed_df

In [133]:
#considero que para el modelo de clasificacion no necesito estas columnas
dfMoviesFinal = dfMoviesFinal.drop(columns=["tagline", "id", "status", "", "budget", 
                                            "release_date","return", "franquicia",
                                            "original_language", "production_country", "language"])

In [134]:
#dropeo estas rows porque considero que tienen generos de pelicula que no me sirven para el modelo, son datos erroneos

dfMoviesFinal.drop(dfMoviesFinal[dfMoviesFinal["movie_genre"]== "Aniplex"].index, inplace=True)
dfMoviesFinal.drop(dfMoviesFinal[dfMoviesFinal["movie_genre"]== "Odyssey Media"].index, inplace=True)
dfMoviesFinal.drop(dfMoviesFinal[dfMoviesFinal["movie_genre"]== "Carousel Productions"].index, inplace=True)

In [135]:
#considero que para poder continuar con el modelo de clasificacion, debo poner el tipo de dato que corresponde a cada columna

strings= ["movie_title", "movie_genre", "overview", "production_company",
            "director", "character", "actor"]
integers=["release_year", "vote_count"]
floats=["runtime", "vote_average", "popularity", "revenue"]

#dropeo valor raro porque sino no puedo ponerle el data type que corresponde
dfMoviesFinal.drop(dfMoviesFinal[dfMoviesFinal['popularity'] == 'Beware Of Frost Bites'].index, inplace=True)

for i in strings:
    dfMoviesFinal[i] = dfMoviesFinal[i].astype("string")
for i in integers:
    dfMoviesFinal[i] = dfMoviesFinal[i].astype("int")
for i in floats:
    dfMoviesFinal[i].replace("", 0, inplace=True) #hay strings vacios en vote_count por eso los lleno con 0
    dfMoviesFinal[i] = dfMoviesFinal[i].astype("float")

In [136]:
#uso feature extraction para trabajar con los resultados del EDA sobre wordclouds

#nube de palabras para la columna title
#palabras mas repetidas: love, day, man, girl

#keywords para que arme las columnas
keywordsTitle = ["day", "man", "girl"]

#origen del texto
text = dfMoviesFinal["movie_title"]

#inicializo la vectorizacion
vectorizer = CountVectorizer(vocabulary=keywordsTitle, lowercase=True)

#cuenta cuantas veces una keyword esta en un movie title
feature = vectorizer.transform(dfMoviesFinal["movie_title"])

#crea df que almacena cuantas veces se cuenta en el titulo de cada pelicula cada feature
feature_dfTitles = pd.DataFrame(feature.toarray(), columns=vectorizer.get_feature_names_out())

#concateno y dropeo la columna movie_title
dfMoviesFinal = pd.concat([feature_dfTitles, dfMoviesFinal], axis=1)
dfMoviesFinal.drop(columns=["movie_title"], inplace=True)


In [137]:
#nube de palabras para la columna overview
#palabras mas repetidas: life, find, love, one

#keywords para que arme las columnas
keywordsOverview = ["life", "find", "love", "one"]

#origen del texto
text = dfMoviesFinal["overview"]

#inicializo la vectorizacion
vectorizer = CountVectorizer(vocabulary=keywordsOverview, lowercase=True)

#cuenta cuantas veces una keyword esta en un movie title
feature = vectorizer.transform(dfMoviesFinal["overview"])

#crea df que almacena cuantas veces se cuenta en el titulo de cada pelicula cada feature
feature_dfOverview = pd.DataFrame(feature.toarray(), columns=vectorizer.get_feature_names_out())

#concateno y dropeo la columna movie_title
dfMoviesFinal = pd.concat([feature_dfOverview, dfMoviesFinal], axis=1)
dfMoviesFinal.drop(columns=["overview"], inplace=True)
dfMoviesFinal

life  find  love  one  day  man  girl         movie_genre  popularity   
0         0     0     0    0    0    0     0           Animation   21.946943  \
1         0     1     0    0    0    0     0           Adventure   17.015539   
2         0     0     0    0    0    0     0             Romance   11.712900   
3         0     1     0    0    0    0     0              Comedy    3.859495   
4         0     0     0    0    0    0     0              Comedy    8.387519   
...     ...   ...   ...  ...  ...  ...   ...                 ...         ...   
50068     0     0     0    0    0    0     0  Genero Desconocido    5.683753   
50069     0     0     0    0    0    0     0  Genero Desconocido    0.178241   
50070     0     0     0    1    0    0     0  Genero Desconocido    0.903007   
50071     1     0     0    3    0    0     0  Genero Desconocido    0.003503   
50072     0     0     0    0    0    0     0  Genero Desconocido    0.163015   

                           production_company  release_year      revenue   
0                     Pixar Animation Studios          1995  373554033.0  \
1                            TriStar Pictures          1995  262797249.0   
2                                Warner Bros.          1995          0.0   
3      Twentieth Century Fox Film Corporation          1995   81452156.0   
4                       Sandollar Productions          1995   76578911.0   
...                                       ...           ...          ...   
50068                                       0          1991          0.0   
50069                                       0          2011          0.0   
50070                                       0          2003          0.0   
50071                                       0          1917          0.0   
50072                                       0          2017          0.0   

       runtime  vote_average  vote_count          director   
0         81.0           7.7        5415     John Lasseter  \
1        104.0           6.9        2413      Joe Johnston   
2        101.0           6.5          92     Howard Deutch   
3        127.0           6.1          34   Forest Whitaker   
4        106.0           5.7         173     Charles Shyer   
...        ...           ...         ...               ...   
50068    104.0           5.7          26        John Irvin   
50069    360.0           9.0           3          Lav Diaz   
50070     90.0           3.8           6    Mark L. Lester   
50071     87.0           0.0           0  Yakov Protazanov   
50072     75.0           0.0           0     Daisy Asquith   

                       character              actor  
0                  Woody (voice)          Tom Hanks  
1                   Alan Parrish     Robin Williams  
2                    Max Goldman     Walter Matthau  
3      Savannah 'Vannah' Jackson    Whitney Houston  
4                   George Banks       Steve Martin  
...                          ...                ...  
50068            Sir Robert Hode     Patrick Bergin  
50069              Sister Angela       Angel Aquino  
50070                 Emily Shaw      Erika Eleniak  
50071                              Iwan Mosschuchin  
50072          Valor Desconocido  Valor Desconocido  

[50073 rows x 18 columns]

In [138]:
#APLICO HASH ENCODING A VARIAS COLUMNAS

hashed_actor =hashEncoding(dfMoviesFinal, "actor", 10)
#concateno y dropeo la columna
dfMoviesFinal = pd.concat([dfMoviesFinal, hashed_actor], axis=1)
dfMoviesFinal.drop(columns=["actor"], inplace=True)

hashed_director =hashEncoding(dfMoviesFinal, "director", 10)
#concateno y dropeo la columna
dfMoviesFinal = pd.concat([dfMoviesFinal, hashed_director], axis=1)
dfMoviesFinal.drop(columns=["director"], inplace=True)

hashed_company =hashEncoding(dfMoviesFinal, "production_company", 10)
#concateno y dropeo la columna
dfMoviesFinal = pd.concat([dfMoviesFinal, hashed_company], axis=1)
dfMoviesFinal.drop(columns=["production_company"], inplace=True)

hashed_character =hashEncoding(dfMoviesFinal, "character", 10)
#concateno y dropeo la columna
dfMoviesFinal = pd.concat([dfMoviesFinal, hashed_character], axis=1)
dfMoviesFinal.drop(columns=["character"], inplace=True)

life  find  love  one  day  man  girl         movie_genre  popularity   
0         0     0     0    0    0    0     0           Animation   21.946943  \
1         0     1     0    0    0    0     0           Adventure   17.015539   
2         0     0     0    0    0    0     0             Romance   11.712900   
3         0     1     0    0    0    0     0              Comedy    3.859495   
4         0     0     0    0    0    0     0              Comedy    8.387519   
...     ...   ...   ...  ...  ...  ...   ...                 ...         ...   
50068     0     0     0    0    0    0     0  Genero Desconocido    5.683753   
50069     0     0     0    0    0    0     0  Genero Desconocido    0.178241   
50070     0     0     0    1    0    0     0  Genero Desconocido    0.903007   
50071     1     0     0    3    0    0     0  Genero Desconocido    0.003503   
50072     0     0     0    0    0    0     0  Genero Desconocido    0.163015   

       release_year  ...  hashed_0  hashed_1  hashed_2  hashed_3  hashed_4   
0              1995  ...       0.0       0.0       0.0       0.0       0.0  \
1              1995  ...       0.0       0.0       0.0       0.0       0.0   
2              1995  ...       0.0       0.0       0.0       0.0       0.0   
3              1995  ...       0.0      -1.0       0.0       0.0       0.0   
4              1995  ...       0.0       0.0       0.0       0.0       0.0   
...             ...  ...       ...       ...       ...       ...       ...   
50068          1991  ...       NaN       NaN       NaN       NaN       NaN   
50069          2011  ...       NaN       NaN       NaN       NaN       NaN   
50070          2003  ...       NaN       NaN       NaN       NaN       NaN   
50071          1917  ...       NaN       NaN       NaN       NaN       NaN   
50072          2017  ...       NaN       NaN       NaN       NaN       NaN   

       hashed_5  hashed_6  hashed_7  hashed_8  hashed_9  
0           0.0       0.0       0.0       0.0       1.0  
1           0.0      -1.0       0.0       0.0       0.0  
2           0.0      -1.0       0.0       0.0       0.0  
3           0.0       0.0       0.0       0.0       0.0  
4           0.0       1.0       0.0       0.0       0.0  
...         ...       ...       ...       ...       ...  
50068       NaN       NaN       NaN       NaN       NaN  
50069       NaN       NaN       NaN       NaN       NaN  
50070       NaN       NaN       NaN       NaN       NaN  
50071       NaN       NaN       NaN       NaN       NaN  
50072       NaN       NaN       NaN       NaN       NaN  

[50073 rows x 54 columns]

In [139]:
#COMIENZO CON EL MODELO DE MACHINE LEARNING

X = 

Index(['life', 'find', 'love', 'one', 'day', 'man', 'girl', 'movie_genre',
       'popularity', 'release_year', 'revenue', 'runtime', 'vote_average',
       'vote_count', 'hashed_0', 'hashed_1', 'hashed_2', 'hashed_3',
       'hashed_4', 'hashed_5', 'hashed_6', 'hashed_7', 'hashed_8', 'hashed_9',
       'hashed_0', 'hashed_1', 'hashed_2', 'hashed_3', 'hashed_4', 'hashed_5',
       'hashed_6', 'hashed_7', 'hashed_8', 'hashed_9', 'hashed_0', 'hashed_1',
       'hashed_2', 'hashed_3', 'hashed_4', 'hashed_5', 'hashed_6', 'hashed_7',
       'hashed_8', 'hashed_9', 'hashed_0', 'hashed_1', 'hashed_2', 'hashed_3',
       'hashed_4', 'hashed_5', 'hashed_6', 'hashed_7', 'hashed_8', 'hashed_9'],
      dtype='object')